In [15]:
%load_ext autoreload
%autoreload 2

import numpy as np
import numpy.random as rd
from scipy import stats
import copy
import os

from pyperplan import planner as pl
from pyperplan import search as sc
from pyperplan.planner import (
    find_domain,
    HEURISTICS,
    search_plan,
    SEARCHES,
    validate_solution,
    write_solution,
)
from pyperplan.task import Operator, Task
from pyperplan.heuristics.heuristic_base import Heuristic
from pyperplan.search.a_star import greedy_best_first_search as gbfs
from typing import Set

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Planning Graph

In [16]:
# define the planning graph
class RelaxedGraph(object):
    
    def __init__(self): # initialise the Graph
        self.num_of_levels: int = 0
        self.act = {0: None}
        self.prop = {}
        self.fixed_point = False
        
            

class RelaxedPlanningGraph(object):
    
    def __init__(self, domain_file: str, problem_file: str):
        # initialise the relaxed planning graph
        self.task = pl._ground(pl._parse(domain_file, problem_file))
        self.graph = None
        self.plan = None
        self.hff = -1 # not yet generated
        self.dom = domain_file
        self.prob = problem_file
        self.success = False # whether successfully generated or not
        
    def create(self, max_level, state = None):
        # create the planning graph with a initial state specified
        # return the level of relaxed graph generated, -1 if reached fixed point, -2 if reached max depth
        self.graph = RelaxedGraph()
        if state is not None:
            self.graph.prop = {0: set(state)}
        else:
            self.graph.prop = {0: set(self.task.initial_state)}
        goal_set = self.task.goals
 
        for level in range(max_level+1):
            current_props = self.graph.prop[level]
            # if the goal has been satisfied
            if Task.goal_reached(self.task, current_props):
                self.success = True
                return level
            
            # else expand the relaxed graph
            self.graph.act[level+1] = set([op for op in self.task.operators if op.applicable(current_props)])
            
            next_props = current_props.copy()
            for op in self.graph.act[level+1]:
                next_props = next_props | op.add_effects
            
            if len(current_props) == len(next_props):
                return -1 # reached fixed point before finding the goal
            self.graph.prop[level+1] = next_props
        
        return -2 #reached max depth
        
    def hff_plan(self):
        # generate a relaxed plan with hff 
        # first return error if graph not successfully generated
        # return hff, -1 if failed
        if self.graph is None:
            print("Graph not yet generated")
            return -1 # graph not yet created
        if not self.success:
            print("Invalid graph")
            return -1 # graph does not reach goal state
        
        # otherwise start backtrace
        # setup g_k
        current_goal = set(self.task.goals.copy())
        k = len(self.graph.act.keys())-1
        self.plan = {}
        for i in range(k, 0, -1):
            act_set = set()
            # select the minimum set of actions that r-satisfied current goal
            for a in self.graph.act[i]:
                for eff in a.add_effects:
                    if eff in current_goal:
                        current_goal.remove(eff)
                        act_set.add(a)
            # update the current goal to be the goals for previous layer
            for a in act_set:
                current_goal.update(a.preconditions)
                
            # update the final plan
            self.plan[i] = list(act_set)

        
        if current_goal.issubset(self.graph.prop[0]):
            count = 0
            for layer in self.plan.values():
                count+=len(layer)
            self.hff = count
            return self.hff
        else:
            print("something went wrong during planning")
            return -1
            
            
        

In [17]:
rpl = RelaxedPlanningGraph("benchmarks/blocks/domain.pddl","benchmarks/blocks/task03.pddl")

rpl.create(999)

graph = rpl.graph

for o in rpl.task.operators:
    print(o.name)

(pick-up d)
(pick-up b)
(pick-up a)
(pick-up c)
(put-down d)
(put-down b)
(put-down a)
(put-down c)
(stack d d)
(stack d b)
(stack d a)
(stack d c)
(stack b d)
(stack b b)
(stack b a)
(stack b c)
(stack a d)
(stack a b)
(stack a a)
(stack a c)
(stack c d)
(stack c b)
(stack c a)
(stack c c)
(unstack d d)
(unstack d b)
(unstack d a)
(unstack d c)
(unstack b d)
(unstack b b)
(unstack b a)
(unstack b c)
(unstack a d)
(unstack a b)
(unstack a a)
(unstack a c)
(unstack c d)
(unstack c b)
(unstack c a)
(unstack c c)


In [18]:
fs = frozenset([6, 7, 8, 9])
s = {1, 2, 3, 4, 5}

y = {1,2,6}

print(s - y) 

{3, 4, 5}


# Feature Vector 

In [93]:
def generate_feature_vec_relaxed(planning_graph, state, max_level):
    """
    Generate the feature vector followed by the paper from the input (problem, state) pair as described by the paper
    Please notice that action and operator are referring to the same type Operator within this function
    
    Inputs
    ----------
    planning_graph: the relaxed planning graph DAG pi
    state: the current state set of facts
    max_level: the maximum layer allowed for ff algorithm to do forward expanding
    
    Outputs
    ----------
    feature_vec: a vector of length n + 2*n**2 + 3 representing the feature generated from the given (problem, state) pair
                 the first n values are single action feature
                 the second 2*n**2 are pairwise action feature
                 the last 3 are original heuristic value, the number of layers in pi and the number of unsatisfied goals
    """
    def get_name(op_name):
        """
        transfer an operator name from format of "(act_name v1 v2...)" to act_name string
        """
        return op_name.split(" ")[0][1:]
    
    
    
    # get action schema and output list
    act_schema = np.array(list(set([get_name(o.name) for o in planning_graph.task.operators]))) # store names of total action schema
    n = len(act_schema)  # length of action schema
    total_len = n+2*n**2+3
    feature_vec = np.zeros(total_len) # return feature vec, first n is linear, second 2*n**2 is pairwise, last 3 is additional feature
    
#     if state == planning_graph.task.goals:
#         return feature_vec # if already the goal
    
    # create and get the graph
    idx = planning_graph.create(max_level, state)
    print(idx)
    planning_graph.hff_plan()
    graph = planning_graph.graph
    act_layers = list(graph.act.values()) # list of layers generated, ith value is the list of actions connencting i-1 th states to ith states layer
    print(planning_graph.task.goals,state)
    # extract linear feature
    #-------------------------------------
    # ith value indicate the num of occurance for ith action of act_schema in the entire graph 
    counter = np.zeros(n)
    for act_layer in act_layers:
        if act_layer is not None:
            for a in act_layer:
                counter[act_schema == get_name(a.name)] += 1 
    feature_vec[0:n] = counter
    
    # extract pair-wise feature
    #-------------------------------------
    # each pair a1, a2 is stored in n + [2*(n*a1+a2), 2*(n*a1+a2)+1]
    # e.g. when a1 is 1, a2 is 3, n is 5, store in 5 + [2*(8),  2*(8)+1]
    
    def to_index(n, index_a1, index_a2, adder):
        """
        return corresponding index in the position of the feature vector
        adder is either 0 or 1
        index_a1, index_a2 refer to move index in act_schema
        """
        return n+2*(n*index_a1+index_a2)+adder
    

    def append_to_dict(a, pre, eff_pos):
        """
        add action a into the dicitonary pre and eff_pos
        """
        for p in a.preconditions:
            current = pre.get(p)
            if current is None:
                current = [a]
            else:
                current.append(a)
            pre[p] = current
            
        for p in a.add_effects:
            current = eff_pos.get(p)
            if current is None:
                current = [a]
            else:
                current.append(a)
            eff_pos[p] = current
            
            return pre, eff_pos


    # define dictionary variables for comparison purpose
    # each dictionary maps a fact(proposition) to a list of actions
    pre = {}
    eff_pos = {}
    
    # add pre and eff into the empty dictionary for the first layer
    for a in act_layers[1]:
        pre, eff_pos = append_to_dict(a, pre, eff_pos)
   
    # loop through second to last action layers
    for i in range(2,len(act_layers)): 
        act_layer = act_layers[i]
        
        # update fecture vec for the entire layer based on all previously visited layers
        for a2 in act_layer:
            # count for num of occurances, use set to avoid multiple counts
            s1 = set() # feature 1 where eff a1 and pre a2 has intersections
            s2 = set() # feature 2 where pre a1 and eff a2 has intersections          
            for p in a2.preconditions:
                current = eff_pos.get(p)
                if current is not None:
                    for a1 in current:
                        s1.add(a1) 

            for p in a2.add_effects:
                current = pre.get(p)
                if current is not None:
                    for a1 in current:
                        s2.add(a1)

            # add index to feature_vec based on set generated:
            index_a2 = int(np.where(get_name(a2.name) == act_schema)[0])
            for a1 in s1:
                # update feature 1 for pair (a1, a2)
                index_a1 = int(np.where(get_name(a1.name) == act_schema)[0])
                feature_vec[to_index(n, index_a1, index_a2,0)]+=1

            for a1 in s2:
                # update feature 2 for pair (a1, a2)
                index_a1 = int(np.where(get_name(a1.name) == act_schema)[0])
                feature_vec[to_index(n, index_a1, index_a2,1)]+=1

        # update pre and eff_pos for the entire layer
        for a2 in act_layer:
            pre, eff_pos = append_to_dict(a2, pre, eff_pos)
           
    # extract final features
    #-------------------------------------
    # add heuristic value, number of layers and number of unsatisfied goals
    # number of layers:
    feature_vec[total_len - 3] = len(act_layers)
    # heuristic value hFF: (number of total actions in the plan)
    feature_vec[total_len - 2] = planning_graph.hff
    # unsatisfied goal: (number of propositions within the initial state that's not in the goal)
    ns_goals = 0
    for fact in state:
        if fact not in planning_graph.task.goals:
            ns_goals += 1
    feature_vec[total_len - 1] = ns_goals
    

    return feature_vec

In [94]:
def find_operator(action : str, ops):
    """
    find an operator from the planning graph's ground operator lists
    
    return: the action operator if found
    """
    for op in ops:
        if op.name == action:
            return op
    return None


def read_plan(plan_file_path: str):
    """
    read all the lines from a plan file directory, remove the last line containing cost
    
    return: a list containing the ground truth plan with length equal to total cost
    """
    with open(plan_file_path, "r") as f:

        # Read the lines of the file into a list of strings
        lines = [line.strip() for line in f.readlines()]

    return lines[:-1]

In [95]:
def generate_training_data(domain_file_path, task_file_path, plan_file_path, problem_num : int):
    """
    generate the feature vector matrix X together with a cost vector y from the given input
    
    Inputs
    ----------
    domain_file_path: the input domain file
    task_file_path: the input problem file
    plan_file_path: the input log file that store the optimal plan
    problem_num: the problem index for this domain
    
    Returns
    ----------
    None, None if no plan can be found (plan has cost 0)
    X : array, shape (plan_length-1, n_features)
        The input feature vec of states from initial states all the way towards the second-last state (one state before goal state)
    Y : array, shape (plan_length-1, 2)
        The input cost vector. If it's a 2D array
        The first column is the true cost pi optimal (assume unit cost)
        The second column is the probelm_num representing the index of this problem
    
    """
    # generate plan and get max level
    plan_actions = read_plan(plan_file_path)
    if len(plan_actions) == 0:
        return None, None
    max_level = len(plan_actions)+2
    
    # generate relaxed planning graph
    planning_graph = RelaxedPlanningGraph(domain_file_path, task_file_path)
    
    # define output matrixes
    X = []
    y = []
    
    # loop from the initial state to the second last state
    current_state = planning_graph.task.initial_state
    current_cost = len(plan_actions)
    for i in range(0,len(plan_actions)-1):
        X.append(generate_feature_vec_relaxed(planning_graph, current_state, max_level))
        y.append(current_cost)
        current_action = find_operator(plan_actions[i], planning_graph.task.operators)
        current_state = current_action.apply(current_state)
        current_cost -=1
        
    y = np.c_[y, problem_num * np.ones(len(y))]
    return np.asarray(X), np.asarray(y)

def generate_problem_matrix(domain_file_path, problem_folder_path, log_folder_path, output_path, title):
    """
    Generate the corresponding feature/label matrix from the given inputs
    Stores in the format of "title.npz" in the output_path
    Each npz file contain two attribute: "feature" and "label"
    
    Inputs
    ----------
    domain_file_path: the path to the domain.pddl
    problem_folder_path: the path to the problem_folder containing all the task problem.pddl for generating vectors
    plan_folder_path: the plan folder that contains all the log files corresponding to each problem task
    output_path: the place to store the generated problem matrix
    title: the name for the output npz file
    """
    # get the training problem names and initialise parameters
    problem_name_list = [f.split('.')[0] for f in os.listdir(problem_folder_path)]
    X = None
    Y = None
    problem_index = 0
    
    # generate the final vectors in X, Y
    for prob in problem_name_list:
        problem_path = problem_folder_path + "/" + prob + ".pddl"
        plan_path = log_folder_path + "/" + prob + "_1800.out"
        temp_X, temp_Y = generate_training_data(domain_file_path, problem_path, plan_path, problem_index)
        if X is None:
            X = temp_X
            Y = temp_Y
        elif temp_X is not None:
            X = np.vstack((X, temp_X))
            Y = np.vstack((Y, temp_Y))
        print(problem_path,X.shape, Y.shape)
        problem_index += 1
       
        
        
    # save the final training vectors
    np.savez(output_path+"/"+title+".npz", feature = X, label = Y)
        

In [96]:
# X, y = generate_training_data("datasets/domain/transport/domain.pddl", 
#                               "datasets/domain/transport/train/instance-1.pddl", 
#                               "datasets/domain/transport/plans/instance-1_1800.out",
#                               0)

In [97]:
generate_problem_matrix("datasets/domain/blocks/domain.pddl", 
                        "datasets/domain/blocks/train", 
                        "datasets/domain/blocks/plans", 
                        "datasets/results", 
                        "blocks-train-10")

2
frozenset({'(on b2 b1)', '(on-table b1)', '(on-table b3)'}) frozenset({'(clear b3)', '(handempty)', '(on b3 b2)', '(on b2 b1)', '(on-table b1)'})
datasets/domain/blocks/train/blocks3-task01.pddl (1, 39) (1, 2)
4
frozenset({'(on-table b3)', '(on-table b2)', '(on b1 b2)'}) frozenset({'(on b2 b3)', '(handempty)', '(on b3 b1)', '(clear b2)', '(on-table b1)'})
4
frozenset({'(on-table b3)', '(on-table b2)', '(on b1 b2)'}) frozenset({'(clear b3)', '(holding b2)', '(on-table b1)', '(on b3 b1)'})
3
frozenset({'(on-table b3)', '(on-table b2)', '(on b1 b2)'}) frozenset({'(clear b3)', '(handempty)', '(on b3 b1)', '(clear b2)', '(on-table b1)', '(on-table b2)'})
3
frozenset({'(on-table b3)', '(on-table b2)', '(on b1 b2)'}) frozenset({'(clear b1)', '(clear b2)', '(on-table b1)', '(holding b3)', '(on-table b2)'})
2
frozenset({'(on-table b3)', '(on-table b2)', '(on b1 b2)'}) frozenset({'(clear b3)', '(clear b1)', '(handempty)', '(clear b2)', '(on-table b1)', '(on-table b3)', '(on-table b2)'})
datase

# RankSVM

In [98]:
import itertools
import numpy as np

from sklearn import svm, linear_model
from sklearn.preprocessing import StandardScaler

def transform_pairwise(X, y):
    """
    Transforms data into pairs for convex relaxation of kendal rank correlation coef
    In this method, all pairs are choosen, except for those that have the same target value or equal cost
    Inputs
    ----------
    X : array, shape (n_samples, n_features)
        The input feature vec of states from of several problems
    y : array, shape (n_samples,) or (n_samples, 2)
        The input cost vector. If it's a 2D array, the second column represents
        the problem index
    Returns
    -------
    X_trans : array, shape (k, n_feaures)
        Difference between features of states (si - sj), only consider the state pair from the same problem
    y_trans : array, shape (k,)
        Output rank labels of values {-1, +1}, 1 represent si has potentially larger cost than sj (further away from goal)
    """
    X_new = []
    y_new = []
    if y.ndim == 1:
        y = np.c_[y, np.ones(y.shape[0])]
    comb = itertools.combinations(range(X.shape[0]), 2)
    for k, (i, j) in enumerate(comb):
        if y[i, 0] == y[j, 0] or y[i, 1] != y[j, 1]:
            # skip if they have the same cost or are from different problem group
            continue
        # otherwise, make the new pair-wise data
        X_new.append(X[i] - X[j])
        y_new.append(np.sign(y[i, 0] - y[j, 0])) # y = 1 if xi further away (larger cost), Vice Vesa
        # randomly output some negative values for training purpose
        if y_new[-1] != (-1) ** k:
            y_new[-1] = - y_new[-1]
            X_new[-1] = - X_new[-1]
    return np.asarray(X_new), np.asarray(y_new)


class RankSVM(svm.LinearSVC):
    """
    Performs pairwise ranking svm with an underlying LinearSVC model
    initialise with a C of regularization term
    default using hinge loss
    """
    
    def __init__(self, C = 1.0):
        super(RankSVM, self).__init__()
        self.C = C
        self.loss = 'hinge'
        self.fit_intercept = False
        self.max_iter = 9999
#         self.scaler = StandardScaler()
        
        
    def fit(self, X, y):
        """
        Fit a pairwise ranking model by first transfer it into pairwise than fitting
        Inputs
        ----------
        X : array, shape (n_samples, n_features)
        y : array, shape (n_samples,) or (n_samples, 2)
        Returns
        -------
        self
        """
        X_trans, y_trans = transform_pairwise(X, y)
        super(RankSVM, self).fit(X_trans, y_trans)
#         X_scaled = self.scaler.fit_transform(X_trans)
#         super(RankSVM, self).fit(X_scaled, y_trans)
        return self

    def predict(self, X):
        """
        Predict an ordering on X. For a list of n samples, this method
        returns a list from 0 to n-1 with the relative order of the rows of X.
        Inputs
        ----------
        X : array, shape (n_samples, n_features)
        Returns
        -------
        rtn: array, shape (n_samples,)
            Returns a list of integers representing the relative order of
            the rows in X.
        """
        if hasattr(self, 'coef_'):
#             return np.argsort(np.dot(self.scaler.transform(X), self.coef_.T).flatten())
            return np.argsort(np.dot(X, self.coef_.T).flatten())
        else:
            raise ValueError("Must call fit() prior to predict()")

    def score(self, X, y):
        """
        Returns the accuracy for the rank prediction, from 0-1
        """
        X_trans, y_trans = transform_pairwise(X, y)
        return np.mean(super(RankSVM, self).predict(X_trans) == y_trans)
    
    def h_val(self, x):
        """
        return pseduo heuristic for search
        """
        if hasattr(self, 'coef_'):
            x_r = x.reshape(1,-1)
#             return np.dot(self.scaler.transform(x_r), self.coef_.T)
            return np.dot(x_r, self.coef_.T)
        else:
            raise ValueError("Must call fit() prior to predict()")
        

# Search

In [99]:
class RankHeuristic(Heuristic):
    """
    Implement the heuristic using the trained RankSVM's coef for dot product
    
    Inadmissible, directly reflect the rank
    
    Default scale value is 10000
    """
    
    def __init__(self, svm : RankSVM, planning_graph, scale_val = 10000):
        super().__init__()
        self.svm = svm
        self.planning_graph = planning_graph
        self.scale = scale_val
        
    def __call__(self, node):
        if (node.state == self.planning_graph.task.goals):
            print(0)
            return 0
        vec = generate_feature_vec_relaxed(self.planning_graph, node.state, 9999999999)
        h = round(self.svm.h_val(vec).item()*self.scale)
        print(h)
        return h

# Training

In [100]:
svm_train = RankSVM(C = 0.1)
results = np.load("datasets/results/blocks-train-10.npz")
X = results['feature']
Y = results['label']
# print(X,Y)
print(X.shape, Y.shape)
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# print(X_scaled.shape, Y.shape)

svm_train.fit(X, Y)

(35, 39) (35, 2)


RankSVM(C=0.1)

In [101]:
print(X[:10],"\n",Y[:10])

print(svm_train.predict(X[1:6]))

# svm_train.score(X[0:23], Y[0:23,0])

# print(np.argsort(Y[23:46,0]))


[[ 2.  0.  1.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  2.  0.  0.  0.  1.  1.  0.
   3.  2.  3.]
 [17.  5.  6. 12.  6. 16.  2.  8.  0.  8. 12. 26.  6. 18.  0.  0.  2.  6.
   0.  3.  8.  0.  3.  3.  0.  0.  8.  8. 14. 41.  3.  0.  1. 14.  5.  4.
   5. 13.  5.]
 [18.  6.  7. 11.  6. 16.  2.  9.  0.  8. 13. 27.  9. 24.  0.  0.  3.  8.
   0.  3.  9.  0.  4.  4.  0.  0.  9.  9. 13. 39.  3.  0.  1. 13.  4.  4.
   5. 14.  4.]
 [15.  6.  5.  8.  6. 12.  2.  6.  0.  6. 10. 22.  9. 24.  0.  0.  3.  8.
   0.  3.  6.  0.  2.  2.  0.  0.  6.  6.  5. 15.  2.  0.  0.  5.  0.  1.
   4.  9.  5.]
 [14.  6.  5.  5.  8.  6.  4.  5.  0.  3. 11. 13.  9. 27.  0.  0.  3.  9.
   0.  2.  4.  0.  2.  2.  0.  0.  5.  5.  6. 18.  2.  0.  0.  6.  0.  0.
   4.  9.  4.]
 [ 9.  6.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  9. 27.  0.  0.  3.  9.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   3.  6.  5.]
 [ 6.  3.  2.  2.  0. 

In [105]:
test_plan = RelaxedPlanningGraph("datasets/domain/blocks/domain.pddl", 
                                 "datasets/domain/blocks/blocks/blocks3/task15.pddl")

rank_h = RankHeuristic(svm_train, test_plan)

# vec = generate_feature_vec_relaxed(test_plan, test_plan.task.goals, 9999999999)
# print(vec)
# h = svm_train.h_val(vec)
# print(h)

gbfs(test_plan.task, rank_h)

2
frozenset({'(on b3 b1)', '(on-table b2)', '(on b1 b2)'}) frozenset({'(clear b3)', '(clear b1)', '(handempty)', '(on b3 b2)', '(on-table b1)', '(on-table b2)'})
36280
3
frozenset({'(on b3 b1)', '(on-table b2)', '(on b1 b2)'}) frozenset({'(clear b3)', '(holding b1)', '(on-table b2)', '(on b3 b2)'})
44919
3
frozenset({'(on b3 b1)', '(on-table b2)', '(on b1 b2)'}) frozenset({'(clear b1)', '(clear b2)', '(on-table b1)', '(holding b3)', '(on-table b2)'})
33801
2
frozenset({'(on b3 b1)', '(on-table b2)', '(on b1 b2)'}) frozenset({'(clear b3)', '(clear b1)', '(handempty)', '(clear b2)', '(on-table b1)', '(on-table b3)', '(on-table b2)'})
23801
2
frozenset({'(on b3 b1)', '(on-table b2)', '(on b1 b2)'}) frozenset({'(clear b3)', '(clear b1)', '(handempty)', '(on b3 b2)', '(on-table b1)', '(on-table b2)'})
36280
3
frozenset({'(on b3 b1)', '(on-table b2)', '(on b1 b2)'}) frozenset({'(clear b3)', '(handempty)', '(on b3 b1)', '(clear b2)', '(on-table b1)', '(on-table b2)'})
49185
3
frozenset({'(on 

IndexError: list index out of range

In [103]:
a = np.array([0,1,2])
print(a.shape)
a = a.reshape(-1,1)
print(a.shape)

(3,)
(3, 1)
